In [19]:
import os 
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")
redis_host = os.getenv("REDIS_HOST")
redis_password = os.getenv('REDIS_PASSWORD')
client = OpenAI()

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=default_model, api_key=api_key)

# InMemory Cache

In [11]:
from langchain_core.messages import HumanMessage
message = [HumanMessage(content = "서울에서 가장 맛있는 길거리 음식 한개만 추천해주세요?")]



In [12]:
%%time
response = llm.invoke(message)  #1번째
print(response.content) 

서울에서 가장 맛있는 길거리 음식으로는 "떡볶이"를 추천합니다. 매콤달콤한 소스에 쫄깃한 떡과 각종 부재료가 어우러져 많은 사람들이 사랑하는 한국의 대표적인 간식입니다. 특히, 명동이나 홍대와 같은 인기 있는 지역에서는 다양한 스타일의 떡볶이를 맛볼 수 있으니 꼭 한 번 시도해보세요!
CPU times: total: 0 ns
Wall time: 2.91 s


In [ ]:
%%time
response = llm.invoke(message)  #2번째
print(response.content) 

서울에서 가장 맛있는 길거리 음식으로는 "떡볶이"를 추천합니다. 매콤달콤한 소스에 쫄깃한 떡과 각종 부재료가 어우러져 많은 사람들이 사랑하는 한국의 대표적인 간식입니다. 특히, 명동이나 홍대와 같은 인기 있는 지역에서는 다양한 스타일의 떡볶이를 맛볼 수 있으니 꼭 한 번 시도해보세요!
CPU times: total: 0 ns
Wall time: 1.51 ms


In [ ]:
# InMemoryCache

from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache  

set_llm_cache(InMemoryCache())

In [30]:
# RedisCache
from langchain_community.cache import RedisCache
from langchain_core.globals import set_llm_cache

import redis

set_llm_cache(RedisCache(redis.Redis(
    host=redis_host,
    port=10480,
    decode_responses=True,
    password=redis_password
)))

message1 = [HumanMessage(content = "이번 시즌 kbo 야구는 lg가 이길까 한화가 이길까?")]
message2 = [HumanMessage(content = "이번주 주말에 데이트하는데 야구를 보는게 좋을까 뮤지컬을 보는게 좋을까?")]


In [31]:
%%time
response = llm.invoke(message1)  #1번째
print(response.content) 

KBO 리그의 경기 결과를 예측하는 것은 어려운 일입니다. LG 트윈스와 한화 이글스는 각기 다른 형태와 전력을 가진 팀들이기 때문에, 그날의 팀 상태, 선수들의 컨디션, 이전 경기의 결과 등이 경기 결과에 큰 영향을 미칩니다. 

최신 경기 성적이나 선수들의 부상 여부, 상황에 따라 승패가 달라질 수 있기 때문에, 이러한 정보를 바탕으로 분석하는 것이 좋습니다. 특정 경기나 시즌에 대한 더 구체적인 정보가 필요하다면, 스포츠 전문 매체나 통계를 참고하는 것이 도움이 될 것입니다.
CPU times: total: 15.6 ms
Wall time: 3.53 s


In [32]:
%%time
response = llm.invoke(message2)  #2번째
print(response.content) 

데이트를 계획하는 건 항상 흥미로운 일이죠! 야구와 뮤지컬 각각의 장단점이 있으니 상황에 따라 다르게 결정할 수 있을 것 같아요.

1. **야구**: 야구는 분위기가 활기차고 서로 함께 응원하며 즐길 수 있는 재미가 있습니다. 날씨가 좋다면 야외에서 소풍처럼 즐기기에도 좋고, 긴장감 있는 순간들을 함께 나누며 소통이 더 잘 될 수 있죠. 

2. **뮤지컬**: 뮤지컬은 감정적으로 몰입할 수 있는 경험을 제공합니다. 아름다운 음악과 공연을 함께 감상하며 서로의 감성을 공유할 수 있는 좋은 기회입니다. 그리고 공연 후에는 그에 대해 이야기하며 더 깊은 대화를 나눌 수 있습니다.

둘 중 어떤 것을 선택하든, 함께하는 시간이 가장 중요하니까 당신과 상대방의 취향을 고려해보세요. 혹시 상대방이 좋아하는 활동이나 최근 보고 싶어했던 공연이 있다면 그걸 중심으로 결정하는 것도 좋은 방법입니다. 즐거운 데이트 되시길 바랍니다!
CPU times: total: 0 ns
Wall time: 5.29 s


# Prompt Template

In [33]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(
    input_variables=["food"],
    template="{food} 레시피 알려줘"
)
prompt = template.format(food="김치찌개")
response = llm.invoke(prompt)
print(prompt)
print(response.content)

김치찌개 레시피 알려줘
김치찌개는 한국의 대표적인 찌개 요리로, 간단하면서도 깊은 맛을 낼 수 있는 요리입니다. 아래에 기본적인 김치찌개 레시피를 소개할게요.

### 재료
- 잘 익은 김치: 2컵 (약 300g)
- 돼지고기 (삼겹살 또는 목살): 150g
- 두부: 1/2 모
- 대파: 1대
- 마늘: 2-3쪽
- 물: 3-4컵
- 고춧가루: 1-2큰술 (취향에 따라 조절)
- 간장: 1큰술
- 소금: 적당량
- 후춧가루: 약간 (선택 사항)

### 만드는 법
1. **재료 손질**:
   - 돼지고기는 한입 크기로 썰고, 두부는 깍둑썰기 합니다.
   - 대파는 송송 썰고, 마늘은 다집니다.

2. **김치와 고기 볶기**:
   - 냄비에 돼지고기를 넣고 중약불로 볶아 기름이 나기 시작하면, 잘 익은 김치를 추가합니다.
   - 김치가 부드러워질 때까지 볶아줍니다.

3. **물 붓기**:
   - 볶은 김치와 고기 위에 물을 붓고 끓입니다. (3~4컵 정도)

4. **양념 추가**:
   - 끓어오르면 고춧가루, 간장, 다진 마늘을 넣고, 중불로 끓입니다.
   - 필요에 따라 소금으로 간을 조절합니다.

5. **두부와 대파 추가**:
   - 김치찌개가 끓어오르면 두부와 대파를 넣고, 10-15분 정도 더 끓입니다.

6. **마무리**:
   - 마지막으로 후춧가루를 약간 넣고, 필요한 경우 소금으로 추가 간을 맞춥니다.

7. **완성**:
   - 그릇에 담아 따뜻할 때 밥과 함께 즐기세요!

김치찌개는 만드는 과정에서 재료의 조합을 조절하여 다양한 맛을 낼 수 있습니다. 기호에 따라 다른 재료를 추가해도 좋습니다! 맛있게 드세요!


In [34]:
prompt = template.format(food="알리올리오")
response = llm.invoke(prompt)
print(prompt)
print(response.content)

알리올리오 레시피 알려줘
알리올리오(aglio e olio)는 이탈리아의 전통 파스타 요리로, 간단하면서도 맛있는 요리입니다. 아래에는 기본적인 알리올리오 레시피를 소개합니다.

### 재료
- 스파게티 200g
- 올리브 오일 4-5 큰술
- 마늘 3-4쪽 (슬라이스 또는 다진 것)
- 빨강 고추 조각 (선택 사항)
- 소금 약간
- 파르미지아노 레지아노 치즈 (선택 사항)
- 파슬리 (선택 사항)

### 조리 방법
1. **파스타 삶기**:
   - 큰 냄비에 물을 넣고 끓인 후, 소금을 충분히 넣습니다.
   - 스파게티를 넣고 포장지에 적힌 시간에 맞춰 알덴테로 삶아줍니다.

2. **마늘 준비**:
   - 다른 팬에 올리브 오일을 넣고 중불로 가열합니다.
   - 슬라이스한 마늘과 빨강 고추 조각을 넣고 천천히 볶습니다. 마늘이 갈색이 되기 시작하면 불을 약하게 줄입니다. (또는 마늘이 타지 않도록 주의)

3. **파스타와 혼합**:
   - 삶은 스파게티를 팬에 옮겨서 마늘과 오일과 잘 섞어줍니다. 필요한 경우, 삶은 물을 조금 추가해가며 섞어줍니다.

4. **마무리**:
   - 소금으로 간을 맞추고, 원한다면 다진 파슬리와 파르미지아노 레지아노 치즈를 위에 뿌려 제공합니다.

### 팁
- 알리올리오는 매우 간단한 요리이므로, 재료의 질이 중요합니다. 좋은 품질의 올리브 오일과 신선한 마늘을 사용하는 것이 좋습니다.
- 매운 맛을 좋아한다면, 고추 조각을 더 많이 추가할 수 있습니다.
- 원하는 경우, 토마토나 브로콜리, 새우 등을 추가하여 변형할 수 있습니다.

맛있게 요리해보세요!


In [36]:
# 프롬프트를 str로 생성,
prompt_str = "{food} 레시피를 알려줘"
template = PromptTemplate.from_template(prompt_str)

prompt = template.format(food="텐동")
response = llm.invoke(prompt)
print(prompt)
print(response.content)

텐동 레시피를 알려줘
텐동(덴푸라 덮밥)은 일본의 대표적인 덮밥 요리 중 하나로, 바삭하게 튀긴 덴푸라와 밥이 함께 제공됩니다. 아래에 간단한 텐동 레시피를 소개할게요.

### 재료
#### 덴푸라 재료
- 새우 8마리
- 고구마 1개
- 가지 1개
- 호박 1/2개
- 밀가루 1컵
- 전분가루 1/4컵
- 얼음물 1컵
- 소금 약간
- 후추 약간
- 튀김油 (식용유) 적당량

#### 밥
- 일본식 쌀 2컵
- 물 2컵

#### 소스
- 간장 1/4컵
- 미irin 1/4컵
- 다시 육수 (또는 물) 1/4컵

### 조리 방법
1. **밥 준비**:
   - 일본식 쌀을 깨끗이 씻어 30분 정도 불려주세요.
   - 물과 함께 밥솥에 넣고 평소처럼 밥을 지어주세요.

2. **재료 손질**:
   - 새우는 껍질을 벗기고 등쪽에 칼집을 내어 뼈를 제거합니다. 살짝 소금과 후추로 간을 해주세요.
   - 고구마, 가지, 호박은 적당한 크기로 썰어줍니다.

3. **튀김 반죽 만들기**:
   - 큰 그릇에 밀가루와 전분가루를 섞어줍니다.
   - 얼음물 (찬 물)을 넣고 가볍게 섞어 반죽을 만듭니다. 반죽은 너무 오래 섞지 말고 덩어리가 있을 정도로만 섞어주세요.

4. **튀김 소스 만들기**:
   - 간장, 미린, 다시 육수를 작은 냄비에 넣고 끓여주세요. 소스가 끓으면 불을 끄고 식혀줍니다.

5. **튀김**:
   - 깊은 프라이팬에 식용유를 넣고 170도 정도로 가열합니다.
   - 새우와 채소에 튀김 반죽을 묻혀서 기름에 넣고 바삭하게 튀겨줍니다. (새우는 약 2-3분, 채소는 약 3-4분 정도 튀깁니다.)
   - 튀겨진 덴푸라는 키친타올에 올려 기름을 빼줍니다.

6. **서빙**:
   - 지은 밥에 튀겨진 덴푸라를 올리고, 준비해 둔 소스를 뿌려줍니다.
   - 취향에 따라 쪽파나 깨를 뿌려서 곁들이면 더욱 맛있습니다.

이제 맛있는 텐동이 완성되었습니다! Enjoy your meal!


In [42]:
# city, adjective, topic
from langchain.prompts import PromptTemplate

prompt_str = "{city}의 {adjective} {topic} 를 찾아줘"
template = PromptTemplate.from_template(prompt_str)
prompt = template.format(city="한국", adjective="볼만한", topic="뮤지컬")
response = llm.invoke(prompt)
print(prompt)
print(response.content)

한국의 볼만한 뮤지컬 를 찾아줘
한국에서 볼만한 뮤지컬은 다양하고, 매년 새로운 작품들이 출연하고 있습니다. 다음은 몇 가지 추천할 만한 뮤지컬입니다:

1. **명성황후** - 한국의 역사적 인물인 명성황후의 이야기를 다룬 작품으로, 웅장한 스토리와 화려한 음악이 특징입니다.

2. **팬텀** - 오페라의 유령을 원작으로 한 뮤지컬로, 감정과 사랑, 그리고 비극적인 요소가 어우러진 작품입니다.

3. **레미제라블** - 빅토르 위고의 소설을 바탕으로 한 뮤지컬로, 사회의 불공평과 인간의 삶에 대한 고뇌를 담고 있습니다.

4. **그리스** - 1950년대 미국의 고등학교를 배경으로 한 뮤지컬로, 젊음과 사랑, 우정을 주제로 한 작품입니다.

5. **지킬앤하이드** - 유명한 소설을 바탕으로 한 작품으로, 인간의 내면의 갈등을 다룬 흥미로운 스토리가 특징입니다.

6. **고스트** - 영화 '고스트'를 원작으로 한 뮤지컬로, 사랑과 상실을 주제로 한 감동적인 이야기입니다.

7. **오페라의 유령** - 앤드류 로이드 웨버의 명작으로, 사랑과 복수, 비극을 다룬 뮤지컬입니다.

각 작품마다 매력이 다르니, 관심 있는 내용을 찾고 예매해 보세요! 뮤지컬 공연 일정은 각 극장 및 제작사의 웹사이트에서 확인할 수 있습니다.


In [39]:
template.save("myTemplate.json")

In [40]:
from langchain.prompts import load_prompt
template = load_prompt

In [43]:
set_llm_cache(None)

- Chat prompt template

In [44]:
from langchain.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 여행 가이드입니다"),
    ("user", "{when}에 {place}로 {travel}을 계획하고 있습니다"),
    ("ai", "내가 바로 {place} 여행 전문가입니다. 무엇이든 물어주세요"),
    ("user", "{question}")
])
print(template)

prompt = template.format_messages(when="이번년도 11월에서 12월달",
                                  place="순천",
                                  travel="여자친구랑 여행",
                                  question="놀거리 추천해줘")

ret = llm.invoke(prompt)
print(ret.content)

input_variables=['place', 'question', 'travel', 'when'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 유능한 여행 가이드입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['place', 'travel', 'when'], input_types={}, partial_variables={}, template='{when}에 {place}로 {travel}을 계획하고 있습니다'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['place'], input_types={}, partial_variables={}, template='내가 바로 {place} 여행 전문가입니다. 무엇이든 물어주세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]
순천은 아름다운 자연과 문화가 어우러진 멋진 여행지입니다. 아래는 순천에서 여자친구와 함께 즐길 수 있는 놀거리와 볼거리를 추천합니다.

1. **순천만 국가정원**: 순천만의 다양한 꽃과 식물을 감상하면서 산책할 수 있는 아름다운 정원입니다. 특히 가을과 겨울철에는 독특한 풍경을 감상할 수 있습니다.

2. **순천만습지**:

- Prompt composition
    sum_prompt = p1 + p2

In [45]:
p1 = PromptTemplate.from_template('오늘은 {topic}에 대해 알려줘')
p2 = PromptTemplate.from_template('{num}개만 알려줘')

sum_prompt = p1 + p2
prompt = sum_prompt.format(topic="서울 맛집", num=3)
ret = llm.invoke(prompt)
print(ret.content)

물론입니다! 서울에는 많은 맛집이 있지만, 그 중에서 추천할 만한 세 곳을 소개해드릴게요.

1. **백리향** (서래마을)
   - 중식 전문점으로, 다양한 중화 요리를 맛볼 수 있습니다. 특히, 북경오리와 딤섬이 유명합니다. 분위기도 좋고, 가족이나 친구와 함께하기 좋은 곳입니다.

2. **김밥천국** (전국 체인)
   - 서울 전역에 지점이 있는 김밥 전문점으로, 가격이 저렴하고 메뉴가 다양합니다. 각종 김밥, 떡볶이, 순대 등을 한 번에 즐길 수 있어 소소한 한 끼로 좋습니다.

3. **을지로 골뱅이** (을지로)
   - 골뱅이 무침과 막걸리가 유명한 포장마차 스타일의 음식점입니다. 학창 시절의 추억을 떠올리게 하는 분위기에서 맛있는 해산물 요리를 즐길 수 있습니다.

이 외에도 서울에는 정말 많은 맛집이 있으니, 다양한 곳을 시도해보시는 것도 좋을 것 같습니다!


- PipelinePromptTemplate

In [46]:
from langchain.prompts.pipeline import PipelinePromptTemplate

final_template = """{prompt_1}

{prompt_2}
"""
final_prompt = PromptTemplate.from_template(final_template)
prompt_1 = PromptTemplate.from_template("당신은 {place} 베테랑 여행 가이드입니다. ")
prompt_2 = PromptTemplate.from_template("고객 최적의 {place} {travel} 일정 수립을 도와줍니다.")
pipeline_prompts = [ 
    ("prompt_1", prompt_1),
    ("prompt_2", prompt_2)
]

pipeline_template = PipelinePromptTemplate(final_prompt=final_prompt, #최종프롬프트 
                                        pipeline_prompts=pipeline_prompts)

prompt = pipeline_template.format(place="동남아", travel="신혼여행")

print(prompt)

당신은 동남아 베테랑 여행 가이드입니다. 

고객 최적의 동남아 신혼여행 일정 수립을 도와줍니다.



C:\Users\edukd\AppData\Local\Temp\ipykernel_3596\1235388041.py:15: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  pipeline_template = PipelinePromptTemplate(final_prompt=final_prompt, #최종프롬프트


- StrOutputParser

In [49]:
from langchain_core.output_parsers import StrOutputParser
prompt = PromptTemplate.from_template("오늘은 {topic}에 대해 알려줘")

template = StrOutputParser()

chain = prompt | llm | template
response = chain.invoke({"topic":"서울 맛집"})
print(response)

서울에는 다양한 맛집이 많아서 선택의 폭이 넓습니다. 이곳은 몇 가지 추천할 만한 맛집을 소개해 드릴게요.

1. **광장시장**
   - 전통시장에서 다양한 먹거리를 즐길 수 있습니다. 빈대떡, 육회, 떡볶이 등 여러 가지 음식을 한자리에서 맛볼 수 있습니다.

2. **이태원**
   - 이태원은 다양한 국제 요리를 즐길 수 있는 곳입니다. 특히 멕시칸, 이탈리안, 인도 요리집이 많이 있어 여러 나라의 음식을 체험할 수 있습니다.

3. **홍대**
   - 젊은 층이 많은 홍대에서는 트렌디한 카페와 맛집이 많습니다. 버거, 파스타, 디저트 카페 등이 인기입니다.

4. **신사동 가로수길**
   - 세련된 분위기의 카페와 레스토랑이 많아 데이트 장소로도 좋습니다. 다양한 퓨전 음식과 디저트를 즐길 수 있습니다.

5. **종로**
   - 전통 한국음식이 유명한 지역으로, 한정식이나 곰탕, 김치찌개 같은 맛있는 한식을 경험할 수 있습니다.

6. **부대찌개**
   - 서울에서 유명한 부대찌개 맛집도 많습니다. 야외에서 친구들과 함께 즐길 수 있는 곳이 많아 인기 있습니다.

7. **김밥천국**
   - 저렴하게 간단한 한식을 즐길 수 있는 패스트푸드 스타일의 식당입니다. 다양한 종류의 김밥과 국을 맛볼 수 있습니다.

서울의 맛집은 정말 다양하니 취향에 맞게 골라보세요! 혹시 특정한 음식이나 분위기를 원하시면 더 구체적으로 추천해 드릴 수 있습니다.


- CommaSeparatedListOutput

In [52]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
format1 = output_parser.get_format_instructions()

prompt = PromptTemplate (
    input_variables=["fruit", "cnt"],
    template="맛있는 {fruit}을 {cnt}개 알려주세요, 이름만 알려주세요, 번호는 적지 마세요",
    partial_variables = {'format_instructions' : format1}
)

res = llm.invoke(prompt.format(fruit="과일", cnt=5))
print(res.content)

print(output_parser.parse(res.content))

사과, 배, Mango, 딸기, 자두
['사과', '배', 'Mango', '딸기', '자두']


1. 모델정의(pydantic)
2. Outputparser정의
3. 포맷 지시문 자동 생성
4. 프롬프드 템플릿 정의
5. 실제 프롬프트 작성
6. 프롬프트 완성
7. LLM 실행(invoke)
8. 파싱처리 
9. 결과 출력

In [53]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# 1. Pydantic 모델 정의
class EmailParser(BaseModel):
    sender_name: str= Field(description="보낸 사람의 이름")
    title: str= Field(description="이메일 제목") 
    content: str= Field(description="이메일 내용")  

# 2. OutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailParser)

# 3. 포맷 지시문 자동 생성
format_instructions = parser.get_format_instructions()

# 4. 프롬프트 템플릿 정의
prompt_template = PromptTemplate.from_template(
    "{format_instructions}\n\n이메일:\n{email}"
)

# 5. 실제 프롬프트 구성
email_text = """보내는 사람: 김철수
제목: 오늘의 회의록
내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."""

prompt = prompt_template.format(
    format_instructions=format_instructions,
    email=email_text
)

# 6. LLM 실행
raw_response = llm.invoke(prompt)

# 7. 파싱 처리
parsed = parser.invoke(raw_response.content)

# 8. 결과 출력
print(parsed)
print(parsed.sender_name)  # 보낸 사람
print(parsed.title)        # 메일 제목
print(parsed.content)      # 회의 내용

sender_name='김철수' title='오늘의 회의록' content='오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'
김철수
오늘의 회의록
오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.


In [55]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class CookingParser(BaseModel):
    ingredient: str= Field(description="재료")
    cooking_order: str= Field(description="요리 순서") 

parser = PydanticOutputParser(pydantic_object=CookingParser)

# 3. 포맷 지시문 자동 생성
format_instructions = parser.get_format_instructions()

# 4. 프롬프트 템플릿 정의
prompt_template = PromptTemplate.from_template(
    "{format_instructions}\n\n요리 레시피를 생각해주세요\n요리 : {food}"
)

# 5. 실제 프롬프트 구성
cooking = """재료 : 고형 카레, 감자, 당근, 양파, 돼지고기, 물, 식용유

순서 : 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."""

prompt = prompt_template.format(
    format_instructions=format_instructions,
    food=cooking
)

# 6. LLM 실행
raw_response = llm.invoke(prompt)

# 7. 파싱 처리
parsed = parser.invoke(raw_response.content)

# 8. 결과 출력
print(parsed)
print(parsed.ingredient)  # 보낸 사람
print(parsed.cooking_order)        # 메일 제목
      # 회의 내용


ingredient='고형 카레, 감자, 당근, 양파, 돼지고기, 물, 식용유' cooking_order='오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'
고형 카레, 감자, 당근, 양파, 돼지고기, 물, 식용유
오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.


In [56]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# --- OpenAI API 키 설정 (실행 전 YOUR_API_KEY를 실제 키로 변경하세요) ---
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 1. LLM 모델 준비
# temperature=0으로 설정하여, 더 예측 가능하고 일관된 결과물을 생성하도록 합니다.
llm = ChatOpenAI(temperature=0, model_name="gpt-4o") 

# 2. 파싱할 데이터 구조 정의 (Pydantic 모델)
# 각 필드에 대한 설명을 명확하게 하여 LLM이 더 정확한 결과를 생성하도록 유도합니다.
class CookingRecipe(BaseModel):
    ingredient: str = Field(description="요리에 필요한 모든 재료를 쉼표로 구분하여 나열합니다.")
    cooking_order: str = Field(description="요리 순서를 단계별로 명확하게 설명합니다.")

# 3. 파서 준비
# Pydantic 모델을 기반으로 출력 파서를 생성합니다.
parser = PydanticOutputParser(pydantic_object=CookingRecipe)

# 4. 프롬프트 템플릿 정의
# LLM에게 어떤 형식으로 답변해야 하는지 명확한 지시사항(format_instructions)을 포함합니다.
prompt_template = PromptTemplate.from_template(
    "아래 주어진 요리 레시피 내용을 분석해서 필요한 정보를 추출해주세요.\n"
    "{format_instructions}\n\n"  # 여기에 파서의 지시사항이 삽입됩니다.
    "## 요리 레시피 내용 ##\n{recipe_text}"
)

# 5. LLM에 입력할 실제 레시피 데이터 준비
# '순서' 부분에 실제 요리 순서를 명확하게 작성해야 합니다.
# 기존의 "회의록" 관련 내용은 잘못된 데이터이므로 수정했습니다.
recipe_input = """
- 재료: 고형 카레 1개, 감자 2개, 당근 1개, 양파 1개, 돼지고기 200g, 물 700ml, 약간의 식용유
- 요리 순서:
  1. 감자, 당근, 양파, 돼지고기를 먹기 좋은 크기로 썬다.
  2. 냄비에 식용유를 두르고 양파가 투명해질 때까지 볶는다.
  3. 돼지고기를 넣고 겉면이 익을 때까지 함께 볶는다.
  4. 나머지 채소(감자, 당근)를 넣고 2분간 더 볶는다.
  5. 물 700ml를 붓고 모든 재료가 부드러워질 때까지 15분간 끓인다.
  6. 불을 끄고 고형 카레를 넣어 완전히 풀어준다.
  7. 다시 약한 불을 켜고 카레가 걸쭉해질 때까지 5분간 더 끓여주면 완성.
"""

# 6. 완성된 프롬프트 생성
# 템플릿에 파서의 지시사항과 레시피 데이터를 결합하여 최종 프롬프트를 만듭니다.
final_prompt = prompt_template.format(
    format_instructions=parser.get_format_instructions(),
    recipe_text=recipe_input
)

# [디버깅] LLM에게 실제로 어떤 프롬프트가 전달되는지 확인해볼 수 있습니다.
# print("------- 최종 프롬프트 -------")
# print(final_prompt)
# print("--------------------------")

# 7. LLM 실행 및 결과 파싱
raw_response = llm.invoke(final_prompt)
parsed_result = parser.invoke(raw_response)

# 8. 최종 결과 출력
print("\n🎉 LLM 응답 파싱 결과 🎉")
print(parsed_result)

print("\n--- 개별 항목 접근 ---")
print(f"✅ 재료: {parsed_result.ingredient}")
print(f"✅ 요리 순서: {parsed_result.cooking_order}")


🎉 LLM 응답 파싱 결과 🎉
ingredient='고형 카레 1개, 감자 2개, 당근 1개, 양파 1개, 돼지고기 200g, 물 700ml, 약간의 식용유' cooking_order='1. 감자, 당근, 양파, 돼지고기를 먹기 좋은 크기로 썬다. 2. 냄비에 식용유를 두르고 양파가 투명해질 때까지 볶는다. 3. 돼지고기를 넣고 겉면이 익을 때까지 함께 볶는다. 4. 나머지 채소(감자, 당근)를 넣고 2분간 더 볶는다. 5. 물 700ml를 붓고 모든 재료가 부드러워질 때까지 15분간 끓인다. 6. 불을 끄고 고형 카레를 넣어 완전히 풀어준다. 7. 다시 약한 불을 켜고 카레가 걸쭉해질 때까지 5분간 더 끓여주면 완성.'

--- 개별 항목 접근 ---
✅ 재료: 고형 카레 1개, 감자 2개, 당근 1개, 양파 1개, 돼지고기 200g, 물 700ml, 약간의 식용유
✅ 요리 순서: 1. 감자, 당근, 양파, 돼지고기를 먹기 좋은 크기로 썬다. 2. 냄비에 식용유를 두르고 양파가 투명해질 때까지 볶는다. 3. 돼지고기를 넣고 겉면이 익을 때까지 함께 볶는다. 4. 나머지 채소(감자, 당근)를 넣고 2분간 더 볶는다. 5. 물 700ml를 붓고 모든 재료가 부드러워질 때까지 15분간 끓인다. 6. 불을 끄고 고형 카레를 넣어 완전히 풀어준다. 7. 다시 약한 불을 켜고 카레가 걸쭉해질 때까지 5분간 더 끓여주면 완성.


In [58]:
template = """요리의 레시피를 생각해주세요
{format_instructions
요리 : {dish}}"""

class Recipe(BaseModel):
    ingredient : list[str] = Field(description="요리 재료")
    recipe : list[str] = Field(description="요리 순서")

parser = PydanticOutputParser(pydantic_object=Recipe)
format_instructions = parser.get_format_instructions()

prompt_template = PromptTemplate.from_template(
    "{format_instructions}\n\n요리:{dish}"
)

prompt = prompt_template.format(
    format_instructions=format_instructions,
    dish ="카레"
)

response = llm.invoke(prompt)
print(response.content)

```json
{
  "ingredient": [
    "닭고기 500g",
    "감자 2개",
    "당근 1개",
    "양파 1개",
    "카레 가루 100g",
    "물 500ml",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간"
  ],
  "recipe": [
    "닭고기를 한 입 크기로 자릅니다.",
    "감자, 당근, 양파를 깍둑썰기 합니다.",
    "팬에 식용유를 두르고 닭고기를 볶습니다.",
    "닭고기가 반쯤 익으면 양파를 넣고 볶습니다.",
    "양파가 투명해지면 감자와 당근을 넣고 볶습니다.",
    "모든 재료가 잘 섞이면 물을 붓고 끓입니다.",
    "물이 끓으면 카레 가루를 넣고 잘 저어줍니다.",
    "약한 불로 줄이고 재료가 익을 때까지 끓입니다.",
    "소금과 후추로 간을 맞춥니다.",
    "완성된 카레를 밥과 함께 제공합니다."
  ]
}
```


In [59]:
myparser = parser.invoke(response.content)
print(myparser)

ingredient=['닭고기 500g', '감자 2개', '당근 1개', '양파 1개', '카레 가루 100g', '물 500ml', '식용유 2큰술', '소금 약간', '후추 약간'] recipe=['닭고기를 한 입 크기로 자릅니다.', '감자, 당근, 양파를 깍둑썰기 합니다.', '팬에 식용유를 두르고 닭고기를 볶습니다.', '닭고기가 반쯤 익으면 양파를 넣고 볶습니다.', '양파가 투명해지면 감자와 당근을 넣고 볶습니다.', '모든 재료가 잘 섞이면 물을 붓고 끓입니다.', '물이 끓으면 카레 가루를 넣고 잘 저어줍니다.', '약한 불로 줄이고 재료가 익을 때까지 끓입니다.', '소금과 후추로 간을 맞춥니다.', '완성된 카레를 밥과 함께 제공합니다.']


In [60]:
print("재료:")
print('\n'.join(myparser.ingredient))
print("\n조리법:")
print('\n'.join(myparser.recipe))

재료:
닭고기 500g
감자 2개
당근 1개
양파 1개
카레 가루 100g
물 500ml
식용유 2큰술
소금 약간
후추 약간

조리법:
닭고기를 한 입 크기로 자릅니다.
감자, 당근, 양파를 깍둑썰기 합니다.
팬에 식용유를 두르고 닭고기를 볶습니다.
닭고기가 반쯤 익으면 양파를 넣고 볶습니다.
양파가 투명해지면 감자와 당근을 넣고 볶습니다.
모든 재료가 잘 섞이면 물을 붓고 끓입니다.
물이 끓으면 카레 가루를 넣고 잘 저어줍니다.
약한 불로 줄이고 재료가 익을 때까지 끓입니다.
소금과 후추로 간을 맞춥니다.
완성된 카레를 밥과 함께 제공합니다.


- LCEL(LangChain Expression Language)

In [62]:
from langchain.chains import SequentialChain, LLMChain

#citydhk transport 는 결과
#landmark는 첫번째 prompt의 결과값으로 사용해야 하므로, 
# 가져올 때 오직 지역의 랜드마크 이름만 가져오도록 만들어야 함
prompt_1 = PromptTemplate.from_template('{city}에서 가장 유명한 랜드마크는 무엇인가요? \\\
                                        설명은 필요없고 딱 이름 하나만 알려주세요')
prompt_2 = PromptTemplate.from_template('{landmark}에 {transport}로 가려면 어떻게 가나요?')

#verbose=True로 하면 결과값을 출력해줌
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='landmark', verbose=True) 
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

chain = SequentialChain(chains=[chain_1, chain_2], input_variables=['city', 'transport'])
chain.invoke({'city':'서울', 'transport':'지하철'})

C:\Users\edukd\AppData\Local\Temp\ipykernel_3596\157183111.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='landmark', verbose=True)




> Entering new LLMChain chain...
Prompt after formatting:
서울에서 가장 유명한 랜드마크는 무엇인가요? \                                        설명은 필요없고 딱 이름 하나만 알려주세요

> Finished chain.


{'city': '서울',
 'transport': '지하철',
 'text': '남산타워(서울타워)에 지하철로 가기 위해서는 몇 가지 경로를 선택할 수 있습니다. 가장 일반적인 방법은 명동역이나 회현역을 이용하는 것입니다.\n\n1. **명동역 경유:**\n   - 지하철 4호선을 타고 명동역에서 하차합니다.\n   - 3번 출구로 나와 남산 케이블카 정류장까지 도보로 약 10분 정도 이동합니다.\n   - 케이블카를 타고 남산타워로 올라갈 수 있습니다.\n\n2. **회현역 경유:**\n   - 지하철 4호선을 타고 회현역에서 하차합니다.\n   - 4번 출구로 나와 남산 오르미 에스컬레이터를 이용해 남산 케이블카 정류장까지 이동합니다.\n   - 케이블카를 타고 남산타워로 올라갈 수 있습니다.\n\n또한, 남산순환버스를 이용할 수도 있습니다. 명동역이나 회현역 근처에서 남산순환버스(02, 03, 05번)를 타고 남산타워 정류장에서 하차하면 됩니다.\n\n각 경로는 출발 위치에 따라 다를 수 있으니, 출발 전에 지하철 노선도와 버스 노선을 확인하시기 바랍니다.'}

In [64]:
from langchain_core.runnables import RunnableLambda
import datetime
def add_date_to_question(question):
    today = datetime.date.today().strftime("%Y년 %m월 %d일")
    return ret
date_runnable = RunnableLambda(add_date_to_question)
prompt = PromptTemplate.from_template("{question}")
chain = date_runnable | prompt | llm
user_question = {"question": "오늘 점심 뭐 먹을까?"}
result = chain.invoke(user_question)
print(result.content)

서울에서 맛집을 찾고 계시다면, 다음 세 곳을 추천드립니다:

1. **백리향** (서래마을)
   - 중식 전문점으로, 다양한 중화 요리를 맛볼 수 있습니다. 특히, 북경오리와 딤섬이 유명합니다. 분위기도 좋고, 가족이나 친구와 함께하기 좋은 곳입니다.

2. **김밥천국** (전국 체인)
   - 서울 전역에 지점이 있는 김밥 전문점으로, 가격이 저렴하고 메뉴가 다양합니다. 각종 김밥, 떡볶이, 순대 등을 한 번에 즐길 수 있어 소소한 한 끼로 좋습니다.

3. **을지로 골뱅이** (을지로)
   - 골뱅이 무침과 막걸리가 유명한 포장마차 스타일의 음식점입니다. 학창 시절의 추억을 떠올리게 하는 분위기에서 맛있는 해산물 요리를 즐길 수 있습니다.

이 외에도 서울에는 정말 많은 맛집이 있으니, 다양한 곳을 시도해보시는 것도 좋을 것 같습니다!


In [67]:
from langchain.agents import load_tools, initialize_agent, AgentType
tools = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(tools,llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("애드 시런은 언제 태어났나요? 지금 몇 살인가요?")



> Entering new AgentExecutor chain...
애드 시런의 생년월일을 알아내기 위해 Wikipedia를 검색해야 합니다. 

Action: wikipedia
Action Input: Ed Sheeran
Observation: Page: Ed Sheeran
Summary: Edward Christopher Sheeran ( SHEER-ən; born 17 February 1991) is an English singer-songwriter. Born in Halifax, West Yorkshire, and raised in Framlingham, Suffolk, he began writing songs around the age of eleven. In early 2011, Sheeran independently released the extended play No. 5 Collaborations Project. He signed with Asylum Records the same year.
Sheeran's debut album, + ("Plus"), was released in September 2011 and topped the UK Albums Chart. It contained his first hit single, "The A Team". In 2012, Sheeran won the Brit Awards for Best British Male Solo Artist and British Breakthrough Act. Sheeran's second studio album, × ("Multiply"), topped charts around the world upon its release in June 2014. It was named the second-best-selling album worldwide of 2015. In the same year, × won Album of the Year at the 2015 Brit Awar

'애드 시런은 1991년 2월 17일에 태어났으며, 현재 32세입니다.'

# 1 UI(Gradio) ->
# 2. msg.submit -> invoke
# 3. 대화에 맥락 만들기 -> history관리 (memory)
# 4. 같은 질문에 API 호출하기 않고, Cache에서 가져오기 (RedisCache)
# --- IGNORE ---

In [ ]:
import gradio as gr
import random
import time

# 응답 생성 함수
def respond(message, history):
    response = random.choice(["안녕하세요", "사랑합니다", "배가 고파요"])
    streamed = ""
    for char in response:
        streamed += char
        time.sleep(0.05)
        yield streamed

# ChatInterface 생성 (불필요한 인자 제거)
demo = gr.ChatInterface(
    fn=respond,
    chatbot=gr.Chatbot(), #채팅 형태의 입출력을 시각적으로 표시하는 컴포넌트
    textbox=gr.Textbox(placeholder="메시지를 입력하세요"),
)

demo.queue()
demo.launch()

C:\Users\edukd\AppData\Local\Temp\ipykernel_3596\1915758248.py:17: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(), #채팅 형태의 입출력을 시각적으로 표시하는 컴포넌트
c:\Users\edukd\miniconda3\envs\myKDA2\Lib\site-packages\gradio\chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [78]:
import gradio as gr       
import random             
import time               

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# 환경설정 API 키 불러오기
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")
redis_host = os.getenv("REDIS_HOST")
redis_password = os.getenv('REDIS_PASSWORD')
client = OpenAI()

llm = ChatOpenAI(model_name=default_model, api_key=api_key, temperature=0.7, streaming=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer the user's questions politely and concisely in Korean."),
    MessagesPlaceholder(variable_name="history"), # 대화 기록이 들어갈 자리
    ("human", "{input}"), # 사용자의 질문이 들어갈 자리
])

# 2-3. 출력 파서(Output Parser) 정의
# LLM의 출력(AIMessage)을 간단한 문자열(string) 형태로 변환합니다.
output_parser = StrOutputParser()

# 2-4. 체인(Chain) 결합
# 프롬프트, 모델, 출력 파서를 '|' (파이프) 연산자로 연결하여 하나의 처리 흐름을 만듭니다. (LCEL 방식)
chain = prompt | llm | output_parser


# --- 3. Gradio 인터페이스와 연결할 함수 정의 ---

def respond(message, history):
    """
    Gradio의 챗봇 인터페이스와 LangChain 체인을 연결하는 함수.
    사용자 메시지와 대화 기록을 받아 스트리밍 응답을 생성합니다.
    """
    # 1. Gradio의 대화 기록 형식을 LangChain이 이해하는 형식으로 변환
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    # 2. 스트리밍 응답 생성
    # chain.stream을 호출하여 LLM으로부터 응답을 실시간으로 받아옵니다.
    stream = chain.stream({
        "input": message,
        "history": history_langchain_format
    })

    # 3. UI에 점진적으로 텍스트 표시
    # history에 사용자 메시지와 AI의 빈 응답칸을 먼저 추가합니다.
    history.append([message, ""])

    # 스트림에서 오는 각 단어(청크)를 AI 응답칸에 계속 추가합니다.
    for chunk in stream:
        history[-1][1] += chunk
        # yield를 통해 변경된 history를 UI에 실시간으로 업데이트합니다.
        # 두 번째 값 ""는 텍스트 입력창을 비우기 위함입니다.
        yield history, ""

        
# Gradio Blocks UI 구성    
# --- 4. Gradio UI 구성 ---

with gr.Blocks(theme=gr.themes.Default()) as demo:
    # CSS 스타일을 적용하여 버튼과 푸터 디자인을 변경합니다.
    gr.HTML("""
    <style>
        #submit-button {
            background-color: #FF7F00 !important; /* 주황색 배경 */
            color: white !important;
            border: none !important;
            font-weight: bold !important;
        }
        #submit-button:hover {
            background-color: #E57200 !important;
        }
        #clear-button {
            background-color: #F0F0F0 !important;
            color: #555 !important;
            border: 1px solid #DDD !important;
        }
        #clear-button:hover {
            background-color: #E0E0E0 !important;
        }
        #footer {
            text-align: center;
            color: #888;
            font-size: 0.9em;
        }
    </style>
    """)
    
    gr.Markdown("LangChain을 이용한 ChatBot")
    
    chatbot = gr.Chatbot(label="대화창", height=500)
    
    msg = gr.Textbox(
        label="메세지 입력",
        placeholder="메시지를 입력하세요"
    )
    
    submit_btn = gr.Button("전송", elem_id="submit-button")
    clear_btn = gr.Button("대화 초기화", elem_id="clear-button")

    gr.Markdown("API를 통해 사용 🎤 Gradio로 제작됨 ⚙️ 설정 ⚙️", elem_id="footer")

    # --- 5. 이벤트 핸들러 연결 ---
    
    # Textbox에서 Enter를 누르거나 '전송' 버튼을 클릭했을 때의 동작 정의
    msg.submit(respond, [msg, chatbot], [chatbot, msg])
    submit_btn.click(respond, [msg, chatbot], [chatbot, msg])
    
    # '초기화' 버튼을 클릭했을 때의 동작 정의
    clear_btn.click(lambda: ([], ""), None, [chatbot, msg])


if __name__ == "__main__":
    # queue()를 사용해 여러 사용자의 동시 요청을 처리할 수 있도록 합니다.
    demo.queue()
    # launch()를 호출하여 웹 앱을 실행합니다.
    demo.launch()

C:\Users\edukd\AppData\Local\Temp\ipykernel_3596\330864697.py:101: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="대화창", height=500)


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
